# 4 - Modeling

In [192]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
import sklearn.metrics as metrics

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils

Using TensorFlow backend.


In [3]:
import pandas as pd
import numpy as np
import re

In [ ]:
# Read in final dataset
df = pd.read_csv('/Users/myokim/Desktop/general_assembly/Projects/Capstone/datasets_clean/final_dataset.csv')

df.drop(columns = ['Unnamed: 0', 'imageAltText', 'salePrice', 'valuePrice', 'skuType'], inplace = True)

In [112]:
df.head()

,brand_name,category,isLimitedEdition,isNew,isOnlineOnly,isSephoraExclusive,listPrice,skuId,target_x,1 star,2 stars,3 stars,4 stars,5 stars,avg_rating,review_count_y,total_ratings,compound_score,compound
0,FENTY BEAUTY by Rihanna,foundation-makeup,False,True,True,True,35.0,2268274,0,0.0,0.0,0.0,5.0,11.0,4.7,16.0,16.0,NaN,0.860733
1,PAT McGRATH LABS,foundation-makeup,False,True,False,True,68.0,2257111,0,28.0,35.0,57.0,212.0,548.0,4.4,880.0,880.0,NaN,0.767496
2,FENTY BEAUTY by Rihanna,foundation-makeup,False,False,False,True,35.0,2164671,0,1139.0,1279.0,1324.0,2149.0,8379.0,4.1,14270.0,14270.0,NaN,0.616840
3,MILK MAKEUP,foundation-makeup,False,True,False,True,36.0,2242105,0,1.0,10.0,23.0,115.0,214.0,4.5,363.0,363.0,NaN,0.775412
4,Estée Lauder,foundation-makeup,False,False,False,False,43.0,2112167,0,213.0,258.0,299.0,837.0,3758.0,4.4,5365.0,5365.0,0.012698,0.782008


In [113]:
df.isnull().sum()

brand_name               0
category                 0
isLimitedEdition         0
isNew                    0
isOnlineOnly             0
isSephoraExclusive       0
listPrice                0
skuId                    0
target_x                 0
1 star                 900
2 stars                900
3 stars                900
4 stars                900
5 stars                900
avg_rating             900
review_count_y         900
total_ratings          900
compound_score        1687
compound               925
dtype: int64

## Feature Engineering

In [114]:
# Create dummy features for binary columns
df = pd.get_dummies(df, columns = ['isLimitedEdition', 'isNew', 'isOnlineOnly', 'isSephoraExclusive', 'brand_name', 'category'], drop_first = True)



df.fillna(0, inplace= True)

## Build Model

### Logistic Regression

In [116]:
X = df.drop(columns = 'target_x')

y = df['target_x']

X_train, X_test, y_train, y_test = train_test_split(X, y,stratify = y, random_state = 42)

In [128]:
y_train.value_counts(normalize = True)

0    0.954686
1    0.045314
Name: target_x, dtype: float64

In [129]:
y_test.value_counts(normalize = True)

0    0.955247
1    0.044753
Name: target_x, dtype: float64

In [137]:
metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'accuracy', 'roc_auc', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'brier_score_loss', 'adjusted_rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted'])

In [179]:
lr = LogisticRegression()
gs_lr = GridSearchCV(
            estimator = lr, 
            param_grid = {
                 'penalty': ['l1','l2'],
                 'C': [.25, .5, .75, 1],
                 'solver': ['liblinear']},
            cv = 5,
            scoring = 'precision')

gs_lr.fit(X_train, y_train)
print(f'Best training score for gs: {gs_lr.best_score_}')
print(f'Best parameters: {gs_lr.best_params_}')
lr = gs_lr.best_estimator_
print(f'Test score: {lr.score(X_test, y_test)}')

/anaconda3/envs/dsi/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/envs/dsi/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/envs/dsi/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/envs/dsi/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/envs/dsi/lib/python3.7/site-packages/sklearn/metrics/

Best training score for gs: 0.3662032269138345
Best parameters: {'C': 0.75, 'penalty': 'l1', 'solver': 'liblinear'}
Test score: 0.9583333333333334


/anaconda3/envs/dsi/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/envs/dsi/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/envs/dsi/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/envs/dsi/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/envs/dsi/lib/python3.7/site-packages/sklearn/metrics/

In [180]:
sum(lr.predict(X_test))

8

In [181]:
sum(lr.predict(X_train))

8

In [150]:
lr_weights = pd.DataFrame(index = X.columns, data = lr.coef_.ravel())

In [222]:
lr_weights[lr_weights[0]>0].sort_values(by = 0,ascending = False)[:5]

,0
brand_name_Black Up,3.539354
brand_name_GLAMGLOW,3.349824
brand_name_trèStiQue,3.108530
brand_name_Wander Beauty,2.435148
brand_name_MILK MAKEUP,1.777935


In [221]:
lr_weights[lr_weights[0]<0].sort_values(by = 0,ascending = True)[:5]

,0
category_mascara,-1.783226
isNew_True,-1.400692
compound,-0.680398
brand_name_tarte,-0.566430
category_setting-powder-face-powder,-0.549702


In [182]:
print(metrics.classification_report(y_test, lr.predict(X_test)))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       619
           1       0.62      0.17      0.27        29

    accuracy                           0.96       648
   macro avg       0.79      0.58      0.62       648
weighted avg       0.95      0.96      0.95       648



In [183]:
cm = confusion_matrix(y_test, lr.predict(X_test))

cm_df = pd.DataFrame(cm, columns = ['pred full price', 'pred sale'],
            index = ['actual full price', 'actual sale'])

In [184]:
cm_df

,pred full price,pred sale
actual full price,616,3
actual sale,24,5


In [212]:
cm_df.iloc[0,0]

616

In [214]:
# Specificity 
(cm_df.iloc[0,0])/(cm_df.iloc[0,1]+cm_df.iloc[0,0])

0.9951534733441034

In [216]:
# Sensitivity 
(cm_df.iloc[1,1]/(cm_df.iloc[1,0]+cm_df.iloc[1,1]))

0.1724137931034483

### Neural Network

In [82]:
ss = StandardScaler()
X_train_sc = ss.fit_transform(X_train)
X_test_sc = ss.transform(X_test)

In [83]:
X_train_sc.shape[1]

181

In [84]:
model = Sequential()

model.add(Dense(32, 
                activation='relu', 
                input_shape=(X_train_sc.shape[1],)))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss = 'binary_crossentropy',
             optimizer = 'adam',
             metrics = ['acc'])

In [85]:
model.fit(X_train_sc, y_train, 
         epochs = 7, 
         batch_size = 128, 
         validation_data = (X_test_sc, y_test))

Train on 1942 samples, validate on 648 samples
Epoch 1/7
1942/1942 [==============================] - 0s 249us/step - loss: 0.5750 - acc: 0.7467 - val_loss: 0.4840 - val_acc: 0.8426
Epoch 2/7
1942/1942 [==============================] - 0s 26us/step - loss: 0.3798 - acc: 0.9083 - val_loss: 0.3410 - val_acc: 0.9151
Epoch 3/7
1942/1942 [==============================] - 0s 28us/step - loss: 0.2752 - acc: 0.9459 - val_loss: 0.2623 - val_acc: 0.9429
Epoch 4/7
1942/1942 [==============================] - 0s 25us/step - loss: 0.2185 - acc: 0.9516 - val_loss: 0.2202 - val_acc: 0.9475
Epoch 5/7
1942/1942 [==============================] - 0s 28us/step - loss: 0.1852 - acc: 0.9542 - val_loss: 0.1948 - val_acc: 0.9475
Epoch 6/7
1942/1942 [==============================] - 0s 26us/step - loss: 0.1643 - acc: 0.9557 - val_loss: 0.1790 - val_acc: 0.9506
Epoch 7/7
1942/1942 [==============================] - 0s 27us/step - loss: 0.1497 - acc: 0.9562 - val_loss: 0.1696 - val_acc: 0.9506


In [60]:
X_train.shape

(1942, 44)

In [61]:
y_test.shape

(648,)

In [92]:
sum(model.predict_classes(X_test))

array([0], dtype=int32)

## Removing brand name from model

In [93]:
# Read in final dataset
df_2 = pd.read_csv('/Users/myokim/Desktop/general_assembly/Projects/Capstone/datasets_clean/final_dataset.csv')

In [94]:
df_2.drop(columns = ['Unnamed: 0', 'imageAltText', 'salePrice', 'valuePrice', 'brand_name', 'skuType'], inplace = True)

In [96]:
# Create dummy features for binary columns
df_2 = pd.get_dummies(df_2, columns = ['isLimitedEdition', 'isNew', 'isOnlineOnly', 'isSephoraExclusive', 'category'], drop_first = True)



In [97]:
df_2.fillna(0, inplace= True)

In [98]:
X = df_2.drop(columns = 'target_x')

In [99]:
y = df_2['target_x']

In [100]:
X_train, X_test, y_train, y_test = train_test_split(X, y,stratify = y)

In [102]:
lr_2 = LogisticRegression()
gs_lr_2 = GridSearchCV(estimator = lr_2, 
             param_grid = {
                 'penalty': ['l1','l2'],
                 'C': [.25, .5, .75, 1],
                 'solver': ['liblinear']},
                    cv = 5)

gs_lr_2.fit(X_train, y_train)
print(f'Best training score for gs: {gs_lr_2.best_score_}')
print(f'Best parameters: {gs_lr_2.best_params_}')
lr_2 = gs_lr_2.best_estimator_
print(f'Test score: {lr_2.score(X_test, y_test)}')

Best training score for gs: 0.9546858908341915
Best parameters: {'C': 0.25, 'penalty': 'l2', 'solver': 'liblinear'}
Test score: 0.9552469135802469


In [103]:
lr_weights_2 = pd.DataFrame(index = X.columns, data = lr.coef_.ravel())

In [104]:
np.percentile(lr_weights_2, .99)

-0.0006398071045870977

In [105]:
lr_weights_2[lr_weights_2[0]>0.0000009].round(5)

,0
isLimitedEdition_True,0.0
isSephoraExclusive_True,0.0


In [106]:
lr_weights_2[lr_weights_2[0]<-0.0003].round(5)

,0
5 stars,-0.00043
review_count_y,-0.00064
total_ratings,-0.00064


In [107]:
sum(lr_2.predict(X_train))

0

In [108]:
sum(y_train)

88

In [109]:
len(y_train)

1942

## Random Forest

Because the classes are so unbalanced, try out random forest

In [185]:
# Read in final dataset
df = pd.read_csv('/Users/myokim/Desktop/general_assembly/Projects/Capstone/datasets_clean/final_dataset.csv')

In [186]:
df.drop(columns = ['Unnamed: 0', 'imageAltText', 'salePrice', 'valuePrice', 'skuType'], inplace = True)

In [187]:
# Create dummy features for binary columns
df = pd.get_dummies(df, columns = ['isLimitedEdition', 'isNew', 'isOnlineOnly', 'isSephoraExclusive', 'brand_name', 'category'], drop_first = True)



In [188]:
df.fillna(0, inplace= True)

In [189]:
X = df.drop(columns = 'target_x')

In [190]:
y = df['target_x']

In [191]:
X_train, X_test, y_train, y_test = train_test_split(X, y,stratify = y, random_state = 42)

In [204]:
rf = RandomForestClassifier()

gs_rf = GridSearchCV(
            estimator = rf, 
            param_grid = {
                'n_estimators': [10,50,100],
                'max_depth': [None, 2],
                'min_samples_split': [2, 5, 8],
                'class_weight':['balanced'],
                'max_features': ['auto', 'log2', None]
            },
            cv = 3)

gs_rf.fit(X_train, y_train)
print(f'Best training score for gs: {gs_rf.best_score_}')
print(f'Best parameters: {gs_rf.best_params_}')
rf = gs_lr.best_estimator_
print(f'Test score: {rf.score(X_test, y_test)}')

Best training score for gs: 0.9536560247167868
Best parameters: {'class_weight': 'balanced', 'max_depth': None, 'max_features': 'auto', 'min_samples_split': 2, 'n_estimators': 10}
Test score: 0.9583333333333334


In [205]:
sum(rf.predict(X_test))

8

In [206]:
sum(rf.predict(X_train))

8

In [207]:
metrics.roc_auc_score(y_test, rf.predict(X_test))

0.5837836332237759

In [208]:
metrics.roc_auc_score(y_test, lr.predict(X_test))

0.5837836332237759

# Log reg round 3

In [232]:
# Read in final dataset
df = pd.read_csv('/Users/myokim/Desktop/general_assembly/Projects/Capstone/datasets_clean/final_dataset.csv')

df.drop(columns = ['Unnamed: 0', 'imageAltText', 'salePrice', 'valuePrice', 'skuType'], inplace = True)

# Create dummy features for binary columns
df = pd.get_dummies(df, columns = ['isLimitedEdition', 'isNew', 'isOnlineOnly', 'isSephoraExclusive', 'brand_name', 'category'], drop_first = True)



df.fillna(0, inplace= True)

X = df.drop(columns = 'target_x')

y = df['target_x']

X_train, X_test, y_train, y_test = train_test_split(X, y,stratify = y, random_state = 42)

In [242]:
lr_3 = LogisticRegression()
gs_lr_3 = GridSearchCV(
            estimator = lr_3, 
            param_grid = {
                 'penalty': ['l1','l2'],
                 'C': [.75, 1],
                 'solver': ['liblinear'],
                 'class_weight': ['balanced'],
                'max_iter' : [5000]},
            cv = 5,
            scoring = 'accuracy')

gs_lr_3.fit(X_train, y_train)
print(f'Best training score for gs: {gs_lr_3.best_score_}')
print(f'Best parameters: {gs_lr_3.best_params_}')
lr_3 = gs_lr_3.best_estimator_
print(f'Test score: {lr_3.score(X_test, y_test)}')

Best training score for gs: 0.8614830072090628
Best parameters: {'C': 0.75, 'class_weight': 'balanced', 'max_iter': 5000, 'penalty': 'l1', 'solver': 'liblinear'}
Test score: 0.8842592592592593


In [247]:
metrics.roc_auc_score(y_test, lr_3.predict(X_test))

0.8736839173305108

In [252]:
print(f'Count of predicted sale test set: {sum(lr_3.predict(X_test))}')

print(f'Count of predicted sale train set: {sum(lr_3.predict(X_train))}')

print(f'Count of actual sales test set: {sum(y_test)}')

Count of predicted sale test set: 96
Count of predicted sale train set: 337
Count of actual sales test set: 29


In [253]:
cm = confusion_matrix(y_test, lr_3.predict(X_test))

cm_df = pd.DataFrame(cm, columns = ['pred full price', 'pred sale'],
            index = ['actual full price', 'actual sale'])

cm_df

,pred full price,pred sale
actual full price,548,71
actual sale,4,25


In [250]:
# Specificity 
(cm_df.iloc[0,0])/(cm_df.iloc[0,1]+cm_df.iloc[0,0])

0.8852988691437803

In [251]:
# Sensitivity 
(cm_df.iloc[1,1]/(cm_df.iloc[1,0]+cm_df.iloc[1,1]))

0.8620689655172413

Change hyperparameters and using scaled data

In [255]:
# Read in final dataset
df = pd.read_csv('/Users/myokim/Desktop/general_assembly/Projects/Capstone/datasets_clean/final_dataset.csv')

df.drop(columns = ['Unnamed: 0', 'imageAltText', 'salePrice', 'valuePrice', 'skuType'], inplace = True)

# Create dummy features for binary columns
df = pd.get_dummies(df, columns = ['isLimitedEdition', 'isNew', 'isOnlineOnly', 'isSephoraExclusive', 'brand_name', 'category'], drop_first = True)



df.fillna(0, inplace= True)

X = df.drop(columns = 'target_x')

y = df['target_x']

X_train, X_test, y_train, y_test = train_test_split(X, y,stratify = y, random_state = 42)

ss = StandardScaler()
X_train_sc = ss.fit_transform(X_train)
X_test_sc = ss.transform(X_test)

In [262]:
lr_3 = LogisticRegression()
gs_lr_3 = GridSearchCV(
            estimator = lr_3, 
            param_grid = {
                 'penalty': ['l1','l2'],
                 'C': [.75, 1],
                 'solver': ['liblinear', 'saga'],
                 'class_weight': ['balanced'],
                'max_iter' : [5000]},
            cv = 5,
            scoring = 'accuracy')

gs_lr_3.fit(X_train_sc, y_train)
print(f'Best training score for gs: {gs_lr_3.best_score_}')
print(f'Best parameters: {gs_lr_3.best_params_}')
lr_3 = gs_lr_3.best_estimator_
print(f'Test score: {lr_3.score(X_test, y_test)}')

Best training score for gs: 0.8594232749742533
Best parameters: {'C': 1, 'class_weight': 'balanced', 'max_iter': 5000, 'penalty': 'l2', 'solver': 'saga'}
Test score: 0.9552469135802469


In [263]:
metrics.roc_auc_score(y_test, lr_3.predict(X_test_sc))

0.8696451451172637

In [264]:
print(f'Count of predicted sale test set: {sum(lr_3.predict(X_test_sc))}')

print(f'Count of predicted sale train set: {sum(lr_3.predict(X_train_sc))}')

print(f'Count of actual sales test set: {sum(y_test)}')

Count of predicted sale test set: 101
Count of predicted sale train set: 340
Count of actual sales test set: 29


In [266]:
cm = confusion_matrix(y_test, lr_3.predict(X_test_sc))

cm_df = pd.DataFrame(cm, columns = ['pred full price', 'pred sale'],
            index = ['actual full price', 'actual sale'])

cm_df

,pred full price,pred sale
actual full price,543,76
actual sale,4,25


In [250]:
# Specificity 
(cm_df.iloc[0,0])/(cm_df.iloc[0,1]+cm_df.iloc[0,0])

0.8852988691437803

In [251]:
# Sensitivity 
(cm_df.iloc[1,1]/(cm_df.iloc[1,0]+cm_df.iloc[1,1]))

0.8620689655172413

In [276]:
lr_3 = LogisticRegression()
gs_lr_3 = GridSearchCV(
            estimator = lr_3, 
            param_grid = {
                 'penalty': ['l1','l2'],
                 'C': [.75, 1],
                 'solver': ['saga'],
                 'class_weight': ['balanced'],
                'max_iter' : [3000,4000,4500]},
            cv = 2,
            scoring = 'accuracy',
            refit = 'roc_auc')

gs_lr_3.fit(X_train_sc, y_train)
print(f'Best training score for gs: {gs_lr_3.best_score_}')
print(f'Best parameters: {gs_lr_3.best_params_}')
lr_3 = gs_lr_3.best_estimator_
print(f'Test score: {lr_3.score(X_test, y_test)}')

/anaconda3/envs/dsi/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Best training score for gs: 0.8759011328527292
Best parameters: {'C': 1, 'class_weight': 'balanced', 'max_iter': 3000, 'penalty': 'l1', 'solver': 'saga'}
Test score: 0.9552469135802469


/anaconda3/envs/dsi/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [277]:
metrics.roc_auc_score(y_test, lr_3.predict(X_test_sc))

0.8712606540025625

In [278]:
print(f'Count of predicted sale test set: {sum(lr_3.predict(X_test_sc))}')

print(f'Count of predicted sale train set: {sum(lr_3.predict(X_train_sc))}')

print(f'Count of actual sales test set: {sum(y_test)}')

Count of predicted sale test set: 99
Count of predicted sale train set: 339
Count of actual sales test set: 29


In [279]:
cm = confusion_matrix(y_test, lr_3.predict(X_test_sc))

cm_df = pd.DataFrame(cm, columns = ['pred full price', 'pred sale'],
            index = ['actual full price', 'actual sale'])

cm_df

,pred full price,pred sale
actual full price,545,74
actual sale,4,25


In [290]:
lr_3 = LogisticRegression()
gs_lr_3 = GridSearchCV(
            estimator = lr_3, 
            param_grid = {
                 'penalty': ['l1'],
                 'C': [1],
                 'solver': ['saga'],
                 'class_weight': [None,'balanced'],
                'max_iter' : [5000, 5500]},
            cv = 2,
            scoring = 'accuracy',
            refit = 'roc_auc')

gs_lr_3.fit(X_train_sc, y_train)
print(f'Best training score for gs: {gs_lr_3.best_score_}')
print(f'Best parameters: {gs_lr_3.best_params_}')
lr_3 = gs_lr_3.best_estimator_
print(f'Test score: {lr_3.score(X_test, y_test)}')

Best training score for gs: 0.9521112255406797
Best parameters: {'C': 1, 'class_weight': None, 'max_iter': 5000, 'penalty': 'l1', 'solver': 'saga'}
Test score: 0.9552469135802469


In [291]:
metrics.roc_auc_score(y_test, lr_3.predict(X_test_sc))

0.6166508829591667

In [292]:
print(f'Count of predicted sale test set: {sum(lr_3.predict(X_test_sc))}')

print(f'Count of predicted sale train set: {sum(lr_3.predict(X_train_sc))}')

print(f'Count of actual sales test set: {sum(y_test)}')

Count of predicted sale test set: 12
Count of predicted sale train set: 28
Count of actual sales test set: 29


In [293]:
cm = confusion_matrix(y_test, lr_3.predict(X_test_sc))

cm_df = pd.DataFrame(cm, columns = ['pred full price', 'pred sale'],
            index = ['actual full price', 'actual sale'])

cm_df

,pred full price,pred sale
actual full price,614,5
actual sale,22,7


In [298]:
lr_3 = LogisticRegression()
gs_lr_3 = GridSearchCV(
            estimator = lr_3, 
            param_grid = {
                 'penalty': ['l1'],
                 'C': [1],
                 'solver': ['saga'],
                 'class_weight': ['balanced'],
                'max_iter' : [5000, 5500]},
            cv = 2,
            scoring = 'precision',
            refit = 'roc_auc')

gs_lr_3.fit(X_train_sc, y_train)
print(f'Best training score for gs: {gs_lr_3.best_score_}')
print(f'Best parameters: {gs_lr_3.best_params_}')
lr_3 = gs_lr_3.best_estimator_
print(f'Test score: {lr_3.score(X_test_sc, y_test)}')

Test score: 0.8796296296296297


In [295]:
metrics.roc_auc_score(y_test, lr_3.predict(X_test_sc))

0.8712606540025625

In [296]:
print(f'Count of predicted sale test set: {sum(lr_3.predict(X_test_sc))}')

print(f'Count of predicted sale train set: {sum(lr_3.predict(X_train_sc))}')

print(f'Count of actual sales test set: {sum(y_test)}')

Count of predicted sale test set: 99
Count of predicted sale train set: 337
Count of actual sales test set: 29


In [297]:
cm = confusion_matrix(y_test, lr_3.predict(X_test_sc))

cm_df = pd.DataFrame(cm, columns = ['pred full price', 'pred sale'],
            index = ['actual full price', 'actual sale'])

cm_df

,pred full price,pred sale
actual full price,545,74
actual sale,4,25
